In [1]:
# The code was removed by Watson Studio for sharing.

## Import Your Dataset

In [2]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007
0,619,2,50,12500,98,1
1,664,0,13,3250,28,1
2,441,1,16,4000,35,1
3,160,2,20,5000,45,1
4,358,1,24,6000,77,0


In [3]:

body = client_887e8fed66994bce85b726a4b6eee3dd.get_object(Bucket='predictblooddonation-donotdelete-pr-zd9e3njxbqzk3r',Key='test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_test = pd.read_csv(body)
df_test.head()



,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation
0,659,2,12,3000,52
1,276,21,7,1750,38
2,263,4,1,250,4
3,303,11,11,2750,38
4,83,4,12,3000,34


In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score, cross_val_predict


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
df_train.shape#check the shape of the data

(576, 6)

In [6]:
np.sum(df_train.duplicated())# no duplicates

0

In [7]:
np.sum(df_test.duplicated())

28

In [8]:
np.sum(df_test.duplicated())

28

In [9]:
#check the null values
df_train.isnull().sum()

Unnamed: 0                     0
Months since Last Donation     0
Number of Donations            0
Total Volume Donated (c.c.)    0
Months since First Donation    0
Made Donation in March 2007    0
dtype: int64

In [10]:
df_test.isnull().sum()

Unnamed: 0                     0
Months since Last Donation     0
Number of Donations            0
Total Volume Donated (c.c.)    0
Months since First Donation    0
dtype: int64

#### No null values

In [11]:
df_train.dtypes

Unnamed: 0                     int64
Months since Last Donation     int64
Number of Donations            int64
Total Volume Donated (c.c.)    int64
Months since First Donation    int64
Made Donation in March 2007    int64
dtype: object

In [12]:
df_train.corr()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007
Unnamed: 0,1.000000,-0.023217,0.063148,0.063148,0.098370,0.041925
Months since Last Donation,-0.023217,1.000000,-0.159731,-0.159731,0.186899,-0.261234
Number of Donations,0.063148,-0.159731,1.000000,1.000000,0.622116,0.220615
Total Volume Donated (c.c.),0.063148,-0.159731,1.000000,1.000000,0.622116,0.220615
Months since First Donation,0.098370,0.186899,0.622116,0.622116,1.000000,-0.019819
Made Donation in March 2007,0.041925,-0.261234,0.220615,0.220615,-0.019819,1.000000


In [13]:
df_train.columns

Index(['Unnamed: 0', 'Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation',
       'Made Donation in March 2007'],
      dtype='object')

In [14]:
ratio=df_train['Months since Last Donation']/df_train['Months since First Donation']
df_train['Ratio']=ratio
df_train.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007,Ratio
0,619,2,50,12500,98,1,0.020408
1,664,0,13,3250,28,1,0.000000
2,441,1,16,4000,35,1,0.028571
3,160,2,20,5000,45,1,0.044444
4,358,1,24,6000,77,0,0.012987


In [15]:
donation_per_month=df_train['Number of Donations']/df_train['Months since First Donation']
df_train['per_month']=donation_per_month
donation_per_month1=df_test['Number of Donations']/df_test['Months since First Donation']
df_test['per_month']=donation_per_month1
avg_don=df_train['Total Volume Donated (c.c.)']/df_train['Months since First Donation']
df_train['avg_don']=avg_don

In [16]:
avg_don=df_test['Total Volume Donated (c.c.)']/df_test['Months since First Donation']
df_test['avg_don']=avg_don
ratio1=df_test['Months since Last Donation']/df_test['Months since First Donation']
df_test['Ratio']=ratio1
df_test.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,per_month,avg_don,Ratio
0,659,2,12,3000,52,0.230769,57.692308,0.038462
1,276,21,7,1750,38,0.184211,46.052632,0.552632
2,263,4,1,250,4,0.250000,62.500000,1.000000
3,303,11,11,2750,38,0.289474,72.368421,0.289474
4,83,4,12,3000,34,0.352941,88.235294,0.117647


In [17]:
df_train.corr()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007,Ratio,per_month,avg_don
Unnamed: 0,1.000000,-0.023217,0.063148,0.063148,0.098370,0.041925,-0.096066,-0.013313,-0.013313
Months since Last Donation,-0.023217,1.000000,-0.159731,-0.159731,0.186899,-0.261234,0.406501,-0.490650,-0.490650
Number of Donations,0.063148,-0.159731,1.000000,1.000000,0.622116,0.220615,-0.529093,0.218207,0.218207
Total Volume Donated (c.c.),0.063148,-0.159731,1.000000,1.000000,0.622116,0.220615,-0.529093,0.218207,0.218207
Months since First Donation,0.098370,0.186899,0.622116,0.622116,1.000000,-0.019819,-0.609268,-0.355318,-0.355318
Made Donation in March 2007,0.041925,-0.261234,0.220615,0.220615,-0.019819,1.000000,-0.214827,0.240273,0.240273
Ratio,-0.096066,0.406501,-0.529093,-0.529093,-0.609268,-0.214827,1.000000,0.073899,0.073899
per_month,-0.013313,-0.490650,0.218207,0.218207,-0.355318,0.240273,0.073899,1.000000,1.000000
avg_don,-0.013313,-0.490650,0.218207,0.218207,-0.355318,0.240273,0.073899,1.000000,1.000000


In [18]:
df_train.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007,Ratio,per_month,avg_don
0,619,2,50,12500,98,1,0.020408,0.510204,127.551020
1,664,0,13,3250,28,1,0.000000,0.464286,116.071429
2,441,1,16,4000,35,1,0.028571,0.457143,114.285714
3,160,2,20,5000,45,1,0.044444,0.444444,111.111111
4,358,1,24,6000,77,0,0.012987,0.311688,77.922078


In [19]:
df_train.corr()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007,Ratio,per_month,avg_don
Unnamed: 0,1.000000,-0.023217,0.063148,0.063148,0.098370,0.041925,-0.096066,-0.013313,-0.013313
Months since Last Donation,-0.023217,1.000000,-0.159731,-0.159731,0.186899,-0.261234,0.406501,-0.490650,-0.490650
Number of Donations,0.063148,-0.159731,1.000000,1.000000,0.622116,0.220615,-0.529093,0.218207,0.218207
Total Volume Donated (c.c.),0.063148,-0.159731,1.000000,1.000000,0.622116,0.220615,-0.529093,0.218207,0.218207
Months since First Donation,0.098370,0.186899,0.622116,0.622116,1.000000,-0.019819,-0.609268,-0.355318,-0.355318
Made Donation in March 2007,0.041925,-0.261234,0.220615,0.220615,-0.019819,1.000000,-0.214827,0.240273,0.240273
Ratio,-0.096066,0.406501,-0.529093,-0.529093,-0.609268,-0.214827,1.000000,0.073899,0.073899
per_month,-0.013313,-0.490650,0.218207,0.218207,-0.355318,0.240273,0.073899,1.000000,1.000000
avg_don,-0.013313,-0.490650,0.218207,0.218207,-0.355318,0.240273,0.073899,1.000000,1.000000


In [20]:
df_train.columns

Index(['Unnamed: 0', 'Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation',
       'Made Donation in March 2007', 'Ratio', 'per_month', 'avg_don'],
      dtype='object')

In [113]:
x=df_train[['Months since Last Donation', 'Number of Donations',
        'Months since First Donation',
         'per_month', 'avg_don'
        ]].values

In [114]:
y=df_train['Made Donation in March 2007'].values

In [115]:
x=preprocessing.StandardScaler().fit(x).transform(x)

#### Split Your Dataset

In [116]:

from sklearn.model_selection import KFold 
kf =KFold(n_splits=10, random_state=4) 

for train_index, test_index in kf.split(x):
      print("Train:", train_index, "Validation:",test_index)
      x_train, x_test = x[train_index], x[test_index] 
      y_train, y_test = y[train_index], y[test_index]

Train: [ 58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129
 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147
 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165
 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183
 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201
 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219
 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237
 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255
 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273
 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291
 292 293 294 295 296 297 298 299 300 301 302

In [117]:
#from sklearn.model_selection import StratifiedKFold
#skf = StratifiedKFold(n_splits=10, random_state=4)
# X is the feature set and y is the target
#for train_index, test_index in skf.split(x,y): 
   # print("Train:", train_index, "Validation:", test_index) 
   # x_train, x_test = x[train_index], x[test_index] 
   # y_train, y_test = y[train_index], y[test_index]

In [118]:
#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=4)

In [119]:
print('x_train',x_train.shape)
print('y_train',y_train.shape)

x_train (519, 5)
y_train (519,)


* Data was split well

In [120]:
model1=DecisionTreeClassifier(criterion='entropy',max_depth=4)
model2=LogisticRegression()
model3=KNeighborsClassifier(n_neighbors=9)

In [121]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(x_train,y_train)
    yhat=neigh.predict(x_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

array([ 0.84210526,  0.92982456,  0.92982456,  0.92982456,  0.92982456,
        0.92982456,  0.92982456,  0.92982456,  0.92982456])

In [30]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1)

The best accuracy was with 0.929824561404 with k= 2


In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
parameters=[{'C':[0.001,0.01,0.1],'random_state':[4],'max_iter':[100,200,300,400]}]
Grid=GridSearchCV(model2,parameters,cv=4)
Grid.fit(x_train,y_train)

GridSearchCV(cv=4, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'random_state': [4], 'max_iter': [100, 200, 300, 400], 'C': [0.001, 0.01, 0.1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
Grid.best_estimator_

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=4, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Train your models

In [122]:
model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='uniform')

### Predict your outcome

In [123]:
y_hat1=model1.predict(x_test)
y_hat2=model2.predict(x_test)
y_hat3=model3.predict(x_test)

In [124]:
print('predicted',y_hat1[0:5])
print('actual',y_test[0:5])

predicted [0 0 0 0 0]
actual [0 0 0 0 0]


#### predict probability

In [125]:
y_hat1prob=model1.predict_proba(x_test)
y_hat2prob=model2.predict_proba(x_test)
y_hat3prob=model3.predict_proba(x_test)

### Evaluation

In [126]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test,y_hat1)

0.89473684210526316

In [127]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test,y_hat2)

0.92982456140350878

In [128]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test,y_hat3)

0.92982456140350878

## Log Loss

In [129]:
from sklearn.metrics import log_loss

In [130]:
log_loss(y_test,y_hat2prob)

0.25380969915513785

In [131]:
log_loss(y_test,y_hat3prob)

0.81320358297938988

### Testing

In [132]:
df_test.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,per_month,avg_don,Ratio
0,659,2,12,3000,52,0.230769,57.692308,0.038462
1,276,21,7,1750,38,0.184211,46.052632,0.552632
2,263,4,1,250,4,0.250000,62.500000,1.000000
3,303,11,11,2750,38,0.289474,72.368421,0.289474
4,83,4,12,3000,34,0.352941,88.235294,0.117647


In [133]:
df_test.columns

Index(['Unnamed: 0', 'Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation',
       'per_month', 'avg_don', 'Ratio'],
      dtype='object')

In [137]:
test_x=df_test[['Months since Last Donation', 'Number of Donations',
        'Months since First Donation',
         'per_month', 'avg_don']].values

In [138]:
test_x=preprocessing.StandardScaler().fit(test_x).transform(test_x)

### predict

In [139]:
yhat1=model1.predict_proba(test_x)
yhat2=model2.predict_proba(test_x)
yhat3=model3.predict_proba(test_x)

In [140]:
yhat1[0:5]

array([[ 0.40740741,  0.59259259],
       [ 0.84090909,  0.15909091],
       [ 0.84      ,  0.16      ],
       [ 1.        ,  0.        ],
       [ 0.40740741,  0.59259259]])

In [141]:
Decision=pd.DataFrame(yhat1)

In [142]:
Decision.head()

,0,1
0,0.407407,0.592593
1,0.840909,0.159091
2,0.840000,0.160000
3,1.000000,0.000000
4,0.407407,0.592593


In [143]:
Decision.columns=['zero','Made Donation in March 2007']

In [144]:
df_final=pd.concat([df_test,Decision],axis=1)
df_final.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,per_month,avg_don,Ratio,zero,Made Donation in March 2007
0,659,2,12,3000,52,0.230769,57.692308,0.038462,0.407407,0.592593
1,276,21,7,1750,38,0.184211,46.052632,0.552632,0.840909,0.159091
2,263,4,1,250,4,0.250000,62.500000,1.000000,0.840000,0.160000
3,303,11,11,2750,38,0.289474,72.368421,0.289474,1.000000,0.000000
4,83,4,12,3000,34,0.352941,88.235294,0.117647,0.407407,0.592593


In [145]:
df_final.columns

Index(['Unnamed: 0', 'Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation',
       'per_month', 'avg_don', 'Ratio', 'zero', 'Made Donation in March 2007'],
      dtype='object')

In [146]:
df_sub=df_final[['Unnamed: 0','Made Donation in March 2007']]
df_sub.head()

,Unnamed: 0,Made Donation in March 2007
0,659,0.592593
1,276,0.159091
2,263,0.160000
3,303,0.000000
4,83,0.592593


In [147]:
logistic=pd.DataFrame(yhat2)
logistic.columns=['zero','Made Donation in March 2007']
logistic.head()

,zero,Made Donation in March 2007
0,0.555109,0.444891
1,0.912917,0.087083
2,0.698391,0.301609
3,0.721141,0.278859
4,0.526684,0.473316


In [148]:
df_final2=pd.concat([df_test,logistic],axis=1)
df_final2.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,per_month,avg_don,Ratio,zero,Made Donation in March 2007
0,659,2,12,3000,52,0.230769,57.692308,0.038462,0.555109,0.444891
1,276,21,7,1750,38,0.184211,46.052632,0.552632,0.912917,0.087083
2,263,4,1,250,4,0.250000,62.500000,1.000000,0.698391,0.301609
3,303,11,11,2750,38,0.289474,72.368421,0.289474,0.721141,0.278859
4,83,4,12,3000,34,0.352941,88.235294,0.117647,0.526684,0.473316


In [149]:
df_sub1=df_final2[['Unnamed: 0','Made Donation in March 2007']]
df_sub1.head()

,Unnamed: 0,Made Donation in March 2007
0,659,0.444891
1,276,0.087083
2,263,0.301609
3,303,0.278859
4,83,0.473316


In [150]:
project.save_data(data=df_sub1.to_csv(index=False),file_name='prediction3.csv',overwrite=True)

{'asset_id': 'edc17afb-438d-4aee-bb55-2bd1d7b49b47',
 'bucket_name': 'predictblooddonation-donotdelete-pr-zd9e3njxbqzk3r',
 'file_name': 'prediction3.csv',
 'message': 'File saved to project storage.'}